In [2]:
ANNOTATION_DIR = '/root/Dataset/COSMOS/cosmos_anns_acm/acm_anns/'
CONTEXT_DIR = '/root/Dataset/COSMOS/context/Context_EL/'

In [5]:
import torch
# from QA import answer

import json
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()

from os.path import join, isfile
from os import listdir
from itertools import groupby
import re
import numpy as np
import random

In [5]:
def get_image_id(filename: str) -> int:
    return int(re.search('([0-9]+)_[0-9]+\.txt', filename).group(1))

context_files = [join(CONTEXT_DIR, file) for file in listdir(CONTEXT_DIR) if isfile(join(CONTEXT_DIR, file))]
context_files = sorted(context_files, key=get_image_id)

context_files_grouped_by_image_id = {key: list(val) for key, val in groupby(context_files, key = get_image_id)}

test_data = pd.read_json(join(ANNOTATION_DIR, 'test_data.json'), lines=True)

In [6]:
test_data['context_filename'] = pd.DataFrame([context_files_grouped_by_image_id]).T
test_data['context_filename'] = [ [] if x is np.NaN else x for x in test_data['context_filename'] ]

In [7]:
def process(row):
    context_filename = row.context_filename
    answers = []
    for filename in context_filename:
        with open(filename, 'r', encoding='utf8') as ifile:
            content = json.load(ifile)
        context = content['context']
        if context == '':
            continue
        context = ' '.join(context.split('\n'))
        answers.append(answer(context, row.caption1, row.caption2))
    return answers

In [10]:
test_data['answers'] = test_data.progress_apply(process, axis=1)

100%|██████████| 1000/1000 [02:04<00:00,  8.02it/s]


In [20]:
test_data.to_csv('./df_answer_EL.csv')

# Stage 2


In [6]:
test_data = pd.read_csv('./df_answer_EL.csv')
test_data.head(3)

,Unnamed: 0.1,Unnamed: 0,img_local_path,caption1,caption2,context_label,article_url,maskrcnn_bboxes,caption1_modified,caption1_entities,caption2_modified,caption2_entities,bert_base_score,bert_large_score,context_filename,answers,sim,pred,final_pred
0,0,0,public_test_mmsys/0.jpg,Julian Castro at his announcement in San Anton...,Julian Castro at his announcement in San Anton...,0,https://www.nytimes.com/2019/06/13/us/politics...,"[[389.9706726074219, 72.9228744506836, 505.056...","PERSON at his announcement in GPE, GPE, on DAT...","[['Julian Castro', 'PERSON'], ['San Antonio', ...","PERSON at his announcement in GPE, GPE, on DATE.","[['Julian Castro', 'PERSON'], ['San Antonio', ...",0.576995,0.601183,['/root/Dataset/COSMOS/context/Context_EL/0_1....,[('lightweight” without the experience needed ...,"[0.06486875, 0.99999994]","[True, False]",NaN
1,1,1,public_test_mmsys/1.jpg,Supporters of Tanzania's ruling Chama Cha Mapi...,A person sits on a truck as supporters of the ...,0,https://www.bbc.com/news/world-africa-54828934,"[[389.6280517578125, 8.949727058410645, 609.61...",Supporters of GPE's ruling ORG party come out ...,"[['Tanzania', 'GPE'], ['Chama Cha Mapinduzi', ...",A person sits on a truck as supporters of the ...,"[['Chama Cha Mapinduzi', 'PERSON'], ['Revoluti...",0.541939,0.729243,['/root/Dataset/COSMOS/context/Context_EL/1_8....,[],[],[],NaN
2,2,2,public_test_mmsys/2.jpg,"53,000 dead people turned up on the state’s vo...",These social media posts did not link to a rec...,1,https://www.snopes.com/fact-check/53000-dead-f...,"[[0.0, 14.214579582214355, 326.70501708984375,...",CARDINAL dead people turned up on the state’s ...,"[['53,000', 'CARDINAL'], ['November 2018', 'DA...",These social media posts did not link to a rec...,"[['Florida', 'GPE'], ['November 2018', 'DATE']...",0.234810,0.307628,['/root/Dataset/COSMOS/context/Context_EL/2_1....,"[('an army of internet trolls', ""53,000 dead p...","[0.025132587, 0.07283948, 0.026980357, 0.10170...","[True, True, True, True, True]",True


In [7]:
from sbert import sbert

def sim(row):
    return [sbert(sentences) for sentences in eval(row.answers) if len(sentences) == 2 ]

test_data['sim'] = test_data.progress_apply(sim, axis=1)

/root/miniconda3/envs/NLP/lib/python3.10/site-packages/transformers/models/auto/modeling_auto.py:1248: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of BertForMaskedLM were not initialized from the model checkpoint at binwang/bert-base-nli-stsb and are newly initialized: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 1000/1000 [02:52<00:00,  5.78it/s]


In [14]:
def infer(row, thresh):
    pred = [i < thresh for i in row.sim]
    return pred

def get_mode(row):
    li = row.pred
    if li == []:
        return 0
    # li = infer(li)
    val, count = np.unique(li, return_counts=True)

    if len(count) == 2 and count[0] == count[1]:
        return 1
    else:
        return val[np.argmax(count)]



In [36]:
from sklearn.metrics import *
prec, recall, acc = [], [], []


thresh = 0.4
test_data['pred'] = test_data.progress_apply(lambda row: infer(row, thresh), axis=1)
test_data['final_pred'] = test_data.progress_apply(get_mode, axis=1)

pred = list(test_data['final_pred'])

ground_truth = list(test_data['context_label'])
for i, x in enumerate(pred):
    if x is None:
        print("HAHA")
        pred[i] = test_data.iloc[i].bert_base_score < 0.5
prec.append(precision_score(ground_truth, pred))
recall.append(recall_score(ground_truth, pred))
acc.append(accuracy_score(ground_truth, pred))
    

100%|██████████| 1000/1000 [00:00<00:00, 27970.42it/s]


In [40]:
acc

[0.597]